In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-conversational-data/intents.json


In [2]:
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=a231a623243a2d6a0ae24c79402372d21119e352b09f6be819ec6959e7db3e0d
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import json
import pickle
import random

**Loading all the content of the intents json file**

In [4]:
#Loading intents.json
with open('/kaggle/input/mental-health-conversational-data/intents.json') as intents:
    data = json.load(intents)

stemmer = LancasterStemmer()

**Extracting all the information from the json file**

In [5]:
All_words = []
All_labels = []
x_words = []
y_words = []

for intent in data['intents']:    #openieng the file 
    for pattern in intent['patterns']:    
        wrds = nltk.word_tokenize(pattern)    #tokeningzing the words 
        All_words.extend(wrds)
        x_words.append(wrds)
        y_words.append(intent['tag'])     #saving it 

        if intent['tag'] not in All_labels:
            All_labels.append(intent['tag'])    #saivng all the tags

In [6]:
data.keys()

dict_keys(['intents'])

In [7]:
# data.values()

# **Let's Preprocess the Data!**

**Sreaming the data and Removing all the duplicates**

In [8]:
All_words = [stemmer.stem(w.lower()) for w in All_words if w not in "?"]
All_words = sorted(list(set(All_words)))
All_labels = sorted(All_labels)

**Preparing the training data by using One hot encoding**

In [9]:
training = []
output = []
out_empty = [0 for _ in range(len(All_labels))]

# One hot encoding
for x, doc in enumerate(x_words):
    Word_bag = []    #creating a bag of words 
    wrds = [stemmer.stem(w) for w in doc]
    for w in All_words:
        if w in wrds:
            Word_bag.append(1)
        else:
            Word_bag.append(0)

    Final_output= out_empty[:]
    Final_output[All_labels.index(y_words[x])] = 1

    training.append(Word_bag)
    output.append(Final_output)


training = np.array(training)
output = np.array(output)

# **Let's Create our Model Uisng Neural Networks!!**

In [10]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 14499  | total loss: 0.16170 | time: 0.086s
| Adam | epoch: 500 | loss: 0.16170 - acc: 0.9591 -- iter: 224/232
Training Step: 14500  | total loss: 0.15165 | time: 0.089s
| Adam | epoch: 500 | loss: 0.15165 - acc: 0.9632 -- iter: 232/232
--


**Yaaaaay!!  Now we need to create a function(just like a bag) for the  predictions**

In [13]:
def bag_of_words(s, All_words):
    Words_bag = [0 for _ in range(len(All_words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(All_words):
            if w == s_word:
                Words_bag[i] = 1

    return np.array(Words_bag)

**Now we need to create a function that ties everything together**

In [14]:
def chat():
    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break
    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, All_words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)
        tag = All_labels[results_index]
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

In [15]:
chat()



You:  hi


Bot:	Hi there. How are you feeling today?




You:  good


Bot:	That's geat to hear. I'm glad you're feeling this way.




You:  what is menatal health


Bot:	Mental health includes our emotional, psychological, and social well-being. It affects how we think, feel, and act. It also helps determine how we handle stress, relate to others, and make choices.




You:  quit
